In [1]:
import warnings
warnings.filterwarnings(action='once')
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from exp.misc import *
from exp.ProcessData import *
from exp.PytorchModels import *
from exp.LearnerClass import *
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
import copy
from torchvision import transforms
import PIL.Image
from sklearn.metrics import roc_auc_score
import torchvision.transforms.functional as TF
from types import MethodType
import sandesh
import pydicom

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got

## Set Parameters

In [2]:
params=json_to_parameters('config.json')
num_folds=5
SEED=220
device = device_by_name('Tesla') # set to GPU name or part of GPU name
model_type='tf_efficientnet_b5_ns' # set model type
loss_int = True # True to use intermidiate in the loss function
pool=True # True - use pooling, False - use linear layer
dropout=0.5
wso_requires_grad=True # True - use WSO, False - static windows
folds=[1]
seed_add=22222
model_version='pdint' #set version name
num_tta=0 # number of augmented features matrixes - 0 means ther will be only 1 an aungemted feature vector per image 
image_type='pkl' #if you unziped train.zip, use 'dicom'
torch.cuda.set_device(device)

In [3]:
df = pd.read_csv(params.path.data+'full_train.csv')


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (23,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
if loss_int:
    WEIGHTS=torch.tensor([0.32929293, 0.01030303, 0.01030303, 0.06060606, 0.15454545,
           0.05151515, 0.04121212, 0.06868687, 0.04121212, 0.06868687,
           0.12363636,0.0404])
    output_size=12
else:
    WEIGHTS=torch.tensor([0.32929293, 0.03030303, 0.03030303, 0.06060606, 0.15454545,
           0.05151515, 0.04121212, 0.06868687, 0.04121212, 0.06868687,
           0.12363636])
    output_size=11

def my_metric(y_pred,y_true,gamma=[0.,0.]):
#     loss=F.binary_cross_entropy_with_logits(torch.tensor(y_pred,dtype=torch.float32),torch.tensor(y_true,dtype=torch.float32),WEIGHTS).detach().item()
    loss=BinaryFocalLoss(gamma=gamma,weights=WEIGHTS)(torch.tensor(y_pred,dtype=torch.float32),torch.tensor(y_true,dtype=torch.float32)).detach().item()
    return_dict={'metric':loss}
    return return_dict


In [ ]:
transform=transforms.Compose([RandomResizedCropTransform(scale=(0.7, 1.3), ratio=(0.7, 1.3)),
                              RotateTransform(45),
                              RandomFlipTransform(0.5,0.5),
                              RandomChangeMeanStdTransform(30,1.),
                              CutoutTransform(0.5,0.2),
                              CutoutTransform(0.25,0.3),
                              ResizeTransform(512,512)])

transform_val=transforms.Compose([SimpleResizeTransform((512,512),-1024)])


## Train

In [ ]:
epoch_in_rep=6
reps =1
num_epochs=epoch_in_rep*reps
batch_size=24
reps_lr=[3e-4*batch_size/32,1e-4*batch_size/32,0.3e-4*batch_size/32]
pos_mul=1
epoch_mul=1
epoch_part=0.15
gamma=[0.,0.]
image_reader=ImageReader(params.path.train,image_type=image_type)
for SEED in [220]:
    val_folds, train_folds, patients_val = create_folds(df,num_folds,SEED)
    for fold in folds: #range(num_folds):
        torch.manual_seed(SEED+fold+seed_add)
        np.random.seed(SEED+fold+seed_add)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        validate_ds=D.Subset(ImageDataset(image_reader,df,transform=transform_val,return_inter=True),val_folds[fold])
        train_ds=D.Subset(ImageDataset(image_reader,df,transform=transform,return_inter=True),train_folds[fold])
        epoch_size=int(len(train_ds)*epoch_part)
        
        model = get_model(model_type,output_size=output_size, feature_size =256,amp=True,pool=pool,
                          dropout=dropout,wso_requires_grad=wso_requires_grad).to(device)
        name=params.model_format.format(model_type,f'image_{SEED}',model_version,fold)
        my_loss=BinaryFocalLoss(gamma=gamma,weights=WEIGHTS.to(device)) 
        learner = Learner(model,None,loss_func=my_loss,name=name,scheduler=None,device=device)
        learner.metric=my_metric
        sweights=np.where(train_ds.dataset.pe_present_on_image[train_folds[fold]],
                  train_ds.dataset.pe_present_on_image[train_folds[fold]],
                  np.sin(train_ds.dataset.rel_slice[train_folds[fold]]*np.pi))+0.2

        learner.sampler=D.WeightedRandomSampler(sweights,epoch_size, replacement=True)
        learner.optimizer = torch.optim.Adam(learner.model.parameters(), lr=1e-4)
        learner.init_amp(do_autocast=False)
        def new_get_y(self,batch):
            return batch[1]
        def new_get_x(self,batch):
            return batch[:1] 
        learner.get_y=MethodType(new_get_y, learner)
        learner.get_x=MethodType(new_get_x, learner)
#        learner.load_checkpoint(params.path.models)


        train_dl_args={'shuffle':False,'batch_size':batch_size,'sampler':learner.sampler}
        val_dl_args={'shuffle':True}
    
        for t in range(reps):
            learner.scheduler = torch.optim.lr_scheduler.OneCycleLR(learner.optimizer, pct_start=0.01,final_div_factor= 10,
                                                                    max_lr=reps_lr[t], steps_per_epoch=epoch_size//batch_size+1, 
                                                                    epochs=num_epochs//reps)

            learner.fit(num_epochs//reps,train_ds,validate_ds,batch_size=batch_size,eval_batch=batch_size,path=params.path.models,
                        train_dl_args=train_dl_args,val_dl_args=val_dl_args,num_workers=12)
        sandesh.send({'name':learner.name,'best_metric':learner.best_metric})     
        learner.save_model(params.path.models)

## Calculate features

In [ ]:
ds0=ImageDataset(image_reader,df,transform=transform_val)
ds=ImageDataset(image_reader,df,transform=transform)
learner.model.last_linear=Noop()
featuress=[learner.predict(ds0,batch_size=64,num_workers=20,do_eval=True)]
#         featuress=[]
sandesh.send({'name':f'finished 1st {SEED} {fold}'})
for i in notebook.tqdm(range(num_tta)):
#                 learner.model=learner.model.train()
    features=learner.predict(ds,batch_size=64,num_workers=12,do_eval=True)
    featuress.append(features)
    sandesh.send({'name':f'finished {i+1} {SEED} {fold}'})

features=np.stack(featuress,0)
with open(params.path.features+(name.split('.')[0]+'.pkl'),'wb') as f:
    pickle.dump(features,f,protocol=4)
sandesh.send({'name':f'saved {features.shape} {SEED} {fold}'})
